# 4.8: Grouping and Aggregating

## Contents:
1. Importing data
2. Finding mean of 'order_number' column grouped by 'department_id'
3. Creating customer loyalty flag
4. Finding mean product prices by customer type
5. Classifying customers' spending
6. Finding frequent vs. non-frequent customers
7. Exporting data


## 1. Importing data

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# creating path for data import

path = r'C:\Users\scott\Desktop\Data Analysis\CF Immersive\Python\Jan-2022 Instacart Basket Analysis'

In [4]:
# importing merged orders/products dataframe

ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'orders_products_labels.pkl'))

## 2. Find mean of 'order_number' column grouped by 'department_id'

In [7]:
# grouping by 'order_number' and finding aggregated mean

df_dept_order_mean = ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

In [15]:
# sorting results descending by mean

df_dept_order_mean.sort_values([('order_number', 'mean')], ascending = False)

,order_number
,mean
department_id,
21,22.902379
10,20.197148
18,19.310397
4,17.811403
16,17.665606
2,17.277920
7,17.225802
19,17.177343


### In the exercise not all of the departments were present, so the results of the entire dataframe are quite different.

## 3. Creating customer loyalty flag

In [16]:
# create max_order column

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [17]:
# Creating flag for loyal customers

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [18]:
# creating flag for regular customers

ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [19]:
# creating flag for new customers

ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [20]:
# checking value counts

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [22]:
# checking output

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer


## 4. Finding mean product prices by customer type

In [26]:
# grouping by 'loyalty_flag' and finding aggregated mean of 'prices'

df_cust_price_mean = ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

In [28]:
# finding columns for sorting

df_cust_price_mean.columns

MultiIndex([('prices', 'mean')],
           )

In [29]:
# sorting results descending by mean

df_cust_price_mean.sort_values([('prices', 'mean')], ascending = False)

,prices
,mean
loyalty_flag,
New customer,13.294670
Regular customer,12.495717
Loyal customer,10.386336


### New customers buy the most expensive products and loyal customers buy the least expenisve products

## 5. Classifying customers' spending

In [30]:
# create avg_price column

ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [32]:
# Creating flag for low spenders

ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [33]:
# Creating flag for high spenders

ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [34]:
# checking output

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,prices,exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,both,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,both,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,both,...,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,both,...,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender


In [35]:
# checking value counts

ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

## 6. Finding frequent vs. non-frequent customers

In [36]:
# create median_days_since column

ords_prods_merge['median_days_since'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [37]:
# Creating flag for Non-frequent customers

ords_prods_merge.loc[ords_prods_merge['median_days_since'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [38]:
# Creating flag for frequent customers

ords_prods_merge.loc[ords_prods_merge['median_days_since'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [39]:
# creating flag for regular customers

ords_prods_merge.loc[(ords_prods_merge['median_days_since'] <= 20) & (ords_prods_merge['median_days_since'] > 10), 'frequency_flag'] = 'Regular customer'

In [40]:
# checking output

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [41]:
# checking value counts

ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

## 7. Exporting data

In [42]:
# exporting new dataframe as .pkl due to its large size

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_agg_Jan_20_2022.pkl'))